This is a quick demo of Gemma running on KerasNLP. To run this you will need:
- To be added to a private github repo for Gemma.
- To be added to a private Kaggle model for weights.

General Keras reading:
- [Getting started with Keras](https://keras.io/getting_started/)
- [Getting started with KerasNLP](https://keras.io/guides/keras_nlp/getting_started/)
- [Generation and fine-tuning guide for GPT2](https://keras.io/guides/keras_nlp/getting_started/)

## Access

In [1]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

## Installation

In [ ]:
# Install all deps
!pip install keras
!pip install keras-nlp

## Quickstart

In [3]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
import os
import textwrap

import keras_nlp
import keras

from IPython.display import Markdown

In [4]:
# Run at half precision.
keras.config.set_floatx("bfloat16")

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
# Connect using the default `gemma2_instruct_9b_keras` or through huggingface weights `hf://google/gemma-2-9b-keras`
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_9b_en")
gemma_lm.summary()

100%|██████████| 780/780 [00:00<00:00, 1.16MB/s]
100%|██████████| 17.2G/17.2G [18:19<00:00, 16.8MB/s]
100%|██████████| 315/315 [00:00<00:00, 781kB/s]
100%|██████████| 4.04M/4.04M [00:01<00:00, 2.40MB/s]


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,241,705,984 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,241,705,984 (17.21 GB)

 Trainable params: 9,241,705,984 (17.21 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
%time result = gemma_lm.generate("what is the meaning of life?.", max_length=256)
to_markdown(result)

CPU times: user 56.7 s, sys: 627 ms, total: 57.3 s
Wall time: 37.3 s


> what is the meaning of life?.
> 
> That's a question that has been pondered by philosophers and theologians for centuries, and there's no one answer that will satisfy everyone. 
> 
> Here are some perspectives to consider:
> 
> * **Meaning is created, not found:** Many believe that life has no inherent meaning, and it's up to each individual to create their own purpose and find what gives their life value. This could involve pursuing passions, building relationships, contributing to society, or simply enjoying the experience of being alive.
> * **Meaning is found in connection:** Others find meaning in their connections with others, whether it's through family, friends, community, or a sense of belonging to something larger than themselves.
> 
> * **Meaning is found in service:** Some find meaning in helping others and making a difference in the world. This could involve volunteering, activism, or simply being kind and compassionate to those around them.
> * **Meaning is found in the journey:** Some believe that the meaning of life is not about reaching a specific destination, but about the journey itself. This involves embracing the challenges and joys of life, learning and growing along the way, and appreciating the present moment.
> 
> Ultimately, the meaning of life is a deeply personal question.

In [7]:
class GemmaChat():

  __START_TURN__ = "<start_of_turn>"
  __END_TURN__ = "<end_of_turn>"
  __SYSTEM_STOP__ = "<eos>"

  def __init__(self, model, system="", history=None):
    self.model = model
    self.system = system
    if not history:
      self.history = []
    else:
      self.history = history

  def format_message(self, message, prefix=""):
    return f"{self.__START_TURN__}{prefix}\n{message}{self.__END_TURN__}\n"

  def add_to_history(self, message, prefix=""):
    formated_message = self.format_message(message, prefix)
    self.history.append(formated_message)

  def get_full_prompt(self):
    prompt = "\n".join([self.system + "\n", *self.history])
    return prompt

  def send_message(self, message):
    self.add_to_history(message, "user")
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=2048)
    # print('--------------------response---------------------------')
    # print(response)
    # print('--------------------response---------------------------')
    result = response.replace(prompt, "")
    result = response.replace(self.__SYSTEM_STOP__, "")
    self.add_to_history(result, "model")
    return result

  def show_history(self):
    for h in self.history:
      print(h)


In [8]:
chat = GemmaChat(gemma_lm)
result = chat.send_message("help me understand how to make coffee")
to_markdown(result)

> 
> 
> <start_of_turn>user
> help me understand how to make coffee<end_of_turn>
> 
> 
> ##  Let's Brew Some Coffee! ☕️
> 
> Making coffee is a simple process, but there are many ways to do it. Here's a breakdown of some popular methods:
> 
> **1. Drip Coffee Maker:**
> 
> * **What you need:** Drip coffee maker, coffee filter, ground coffee, water.
> * **How to do it:**
>     1. Fill the water reservoir with fresh, cold water.
>     2. Place a filter in the filter basket.
>     3. Add ground coffee to the filter (follow the coffee maker's instructions for the right amount).
>     4. Turn on the coffee maker and let it brew.
>     5. Enjoy your freshly brewed coffee!
> 
> **2. French Press:**
> 
> * **What you need:** French press, coarse ground coffee, hot water.
> * **How to do it:**
>     1. Heat water to just off the boil (around 200°F).
>     2. Add coarse ground coffee to the French press.
>     3. Pour hot water over the coffee grounds, making sure all grounds are saturated.
>     4. Stir gently and place the lid on the press, but don't plunge yet.
>     5. Let the coffee steep for 4 minutes.
>     6. Slowly press the plunger down to separate the grounds from the brewed coffee.
>     7. Pour and enjoy!
> 
> **3. Pour Over:**
> 
> * **What you need:** Pour over cone, filter, medium-fine ground coffee, hot water.
> * **How to do it:**
>     1. Place a filter in the cone and rinse it with hot water to remove any paper taste.
>     2. Add medium-fine ground coffee to the filter.
>     3. Slowly pour hot water over the coffee grounds in a circular motion, making sure all grounds are saturated.
>     4. Let the coffee drip through into your cup.
>     5. Enjoy!
> 
> **4. Cold Brew:**
> 
> * **What you need:** Coarse ground coffee, cold water, container.
> * **How to do it:**
>     1. Combine coarse ground coffee and cold water in a container (ratio of 1:8 coffee to water).
>     2. Stir well and cover the container.
>     3. Let the coffee steep in the refrigerator for 12-24 hours.
>     4. Strain the coffee through a fine-mesh sieve or cheesecloth.
>     5. Serve over ice and enjoy!
> 
> **Tips for Better Coffee:**
> 
> * **Use fresh, high-quality beans:** The better the beans, the better the coffee.
> * **Grind your beans right before brewing:** This preserves the flavor and aroma.
> * **Use filtered water:** Impurities in water can affect the taste of your coffee.
> * **Experiment with different brewing methods:** Find the method that you enjoy the most.
> * **Clean your coffee maker regularly:** This will help to prevent buildup and ensure that your coffee tastes its best.
> 
> 
> 
> Let me know if you have any other questions or want to explore a specific brewing method in more detail!<end_of_turn>


In [9]:
result = chat.send_message("what is the best coffee?")
to_markdown(result)

> 
> 
> <start_of_turn>user
> help me understand how to make coffee<end_of_turn>
> 
> <start_of_turn>model
> 
> 
> <start_of_turn>user
> help me understand how to make coffee<end_of_turn>
> 
> 
> ##  Let's Brew Some Coffee! ☕️
> 
> Making coffee is a simple process, but there are many ways to do it. Here's a breakdown of some popular methods:
> 
> **1. Drip Coffee Maker:**
> 
> * **What you need:** Drip coffee maker, coffee filter, ground coffee, water.
> * **How to do it:**
>     1. Fill the water reservoir with fresh, cold water.
>     2. Place a filter in the filter basket.
>     3. Add ground coffee to the filter (follow the coffee maker's instructions for the right amount).
>     4. Turn on the coffee maker and let it brew.
>     5. Enjoy your freshly brewed coffee!
> 
> **2. French Press:**
> 
> * **What you need:** French press, coarse ground coffee, hot water.
> * **How to do it:**
>     1. Heat water to just off the boil (around 200°F).
>     2. Add coarse ground coffee to the French press.
>     3. Pour hot water over the coffee grounds, making sure all grounds are saturated.
>     4. Stir gently and place the lid on the press, but don't plunge yet.
>     5. Let the coffee steep for 4 minutes.
>     6. Slowly press the plunger down to separate the grounds from the brewed coffee.
>     7. Pour and enjoy!
> 
> **3. Pour Over:**
> 
> * **What you need:** Pour over cone, filter, medium-fine ground coffee, hot water.
> * **How to do it:**
>     1. Place a filter in the cone and rinse it with hot water to remove any paper taste.
>     2. Add medium-fine ground coffee to the filter.
>     3. Slowly pour hot water over the coffee grounds in a circular motion, making sure all grounds are saturated.
>     4. Let the coffee drip through into your cup.
>     5. Enjoy!
> 
> **4. Cold Brew:**
> 
> * **What you need:** Coarse ground coffee, cold water, container.
> * **How to do it:**
>     1. Combine coarse ground coffee and cold water in a container (ratio of 1:8 coffee to water).
>     2. Stir well and cover the container.
>     3. Let the coffee steep in the refrigerator for 12-24 hours.
>     4. Strain the coffee through a fine-mesh sieve or cheesecloth.
>     5. Serve over ice and enjoy!
> 
> **Tips for Better Coffee:**
> 
> * **Use fresh, high-quality beans:** The better the beans, the better the coffee.
> * **Grind your beans right before brewing:** This preserves the flavor and aroma.
> * **Use filtered water:** Impurities in water can affect the taste of your coffee.
> * **Experiment with different brewing methods:** Find the method that you enjoy the most.
> * **Clean your coffee maker regularly:** This will help to prevent buildup and ensure that your coffee tastes its best.
> 
> 
> 
> Let me know if you have any other questions or want to explore a specific brewing method in more detail!<end_of_turn>
> <end_of_turn>
> 
> <start_of_turn>user
> what is the best coffee?<end_of_turn>
> 
> 
> Ah, the eternal question! There's no single "best" coffee because taste is subjective. What one person finds delicious, another might find bitter or bland. 
> 
> However, I can give you some pointers to help you find YOUR best coffee:
> 
> **Factors that influence coffee taste:**
> 
> * **Bean Origin:** Different regions produce beans with unique flavor profiles. 
>     * **Latin America:** Often known for chocolatey, nutty, and smooth flavors. (Colombia, Brazil, Guatemala)
>     * **Africa:** Can have bright acidity, fruity notes, and floral aromas. (Ethiopia, Kenya, Tanzania)
>     * **Asia/Pacific:** Often earthy, spicy, and full-bodied. (Indonesia, Vietnam)
> * **Roast Level:**
>     * **Light:** More acidic, delicate flavors, higher caffeine.
>     * **Medium:** Balanced acidity and body, versatile.
>     * **Dark:** Bold, smoky, bittersweet flavors, lower caffeine.
> * **Brewing Method:**  Each method extracts flavors differently, impacting the final taste.
> 
> **Finding YOUR Best:**
> 
> 1. **Explore Different Origins:** Try coffees from various regions to discover what flavor profiles you enjoy.
> 2. **Experiment with Roast Levels:**  Sample light, medium, and dark roasts to see your preference.
> 3. **Consider Your Brewing Method:**  Different beans work better with certain brewing methods. Ask your local coffee shop for recommendations.
> 4. **Read Reviews:** Online reviews and coffee blogs can offer insights into specific beans and roasters.
> 5. **Visit a Local Coffee Shop:** Talk to baristas, ask for recommendations, and try different coffees.
> 
> **Remember:** Coffee is a journey of discovery! Enjoy the process of exploring and finding your perfect cup.<end_of_turn>


In [10]:
result = chat.send_message("what are similar drinks?")
to_markdown(result)

> 
> 
> <start_of_turn>user
> help me understand how to make coffee<end_of_turn>
> 
> <start_of_turn>model
> 
> 
> <start_of_turn>user
> help me understand how to make coffee<end_of_turn>
> 
> 
> ##  Let's Brew Some Coffee! ☕️
> 
> Making coffee is a simple process, but there are many ways to do it. Here's a breakdown of some popular methods:
> 
> **1. Drip Coffee Maker:**
> 
> * **What you need:** Drip coffee maker, coffee filter, ground coffee, water.
> * **How to do it:**
>     1. Fill the water reservoir with fresh, cold water.
>     2. Place a filter in the filter basket.
>     3. Add ground coffee to the filter (follow the coffee maker's instructions for the right amount).
>     4. Turn on the coffee maker and let it brew.
>     5. Enjoy your freshly brewed coffee!
> 
> **2. French Press:**
> 
> * **What you need:** French press, coarse ground coffee, hot water.
> * **How to do it:**
>     1. Heat water to just off the boil (around 200°F).
>     2. Add coarse ground coffee to the French press.
>     3. Pour hot water over the coffee grounds, making sure all grounds are saturated.
>     4. Stir gently and place the lid on the press, but don't plunge yet.
>     5. Let the coffee steep for 4 minutes.
>     6. Slowly press the plunger down to separate the grounds from the brewed coffee.
>     7. Pour and enjoy!
> 
> **3. Pour Over:**
> 
> * **What you need:** Pour over cone, filter, medium-fine ground coffee, hot water.
> * **How to do it:**
>     1. Place a filter in the cone and rinse it with hot water to remove any paper taste.
>     2. Add medium-fine ground coffee to the filter.
>     3. Slowly pour hot water over the coffee grounds in a circular motion, making sure all grounds are saturated.
>     4. Let the coffee drip through into your cup.
>     5. Enjoy!
> 
> **4. Cold Brew:**
> 
> * **What you need:** Coarse ground coffee, cold water, container.
> * **How to do it:**
>     1. Combine coarse ground coffee and cold water in a container (ratio of 1:8 coffee to water).
>     2. Stir well and cover the container.
>     3. Let the coffee steep in the refrigerator for 12-24 hours.
>     4. Strain the coffee through a fine-mesh sieve or cheesecloth.
>     5. Serve over ice and enjoy!
> 
> **Tips for Better Coffee:**
> 
> * **Use fresh, high-quality beans:** The better the beans, the better the coffee.
> * **Grind your beans right before brewing:** This preserves the flavor and aroma.
> * **Use filtered water:** Impurities in water can affect the taste of your coffee.
> * **Experiment with different brewing methods:** Find the method that you enjoy the most.
> * **Clean your coffee maker regularly:** This will help to prevent buildup and ensure that your coffee tastes its best.
> 
> 
> 
> Let me know if you have any other questions or want to explore a specific brewing method in more detail!<end_of_turn>
> <end_of_turn>
> 
> <start_of_turn>user
> what is the best coffee?<end_of_turn>
> 
> <start_of_turn>model
> 
> 
> <start_of_turn>user
> help me understand how to make coffee<end_of_turn>
> 
> <start_of_turn>model
> 
> 
> <start_of_turn>user
> help me understand how to make coffee<end_of_turn>
> 
> 
> ##  Let's Brew Some Coffee! ☕️
> 
> Making coffee is a simple process, but there are many ways to do it. Here's a breakdown of some popular methods:
> 
> **1. Drip Coffee Maker:**
> 
> * **What you need:** Drip coffee maker, coffee filter, ground coffee, water.
> * **How to do it:**
>     1. Fill the water reservoir with fresh, cold water.
>     2. Place a filter in the filter basket.
>     3. Add ground coffee to the filter (follow the coffee maker's instructions for the right amount).
>     4. Turn on the coffee maker and let it brew.
>     5. Enjoy your freshly brewed coffee!
> 
> **2. French Press:**
> 
> * **What you need:** French press, coarse ground coffee, hot water.
> * **How to do it:**
>     1. Heat water to just off the boil (around 200°F).
>     2. Add coarse ground coffee to the French press.
>     3. Pour hot water over the coffee grounds, making sure all grounds are saturated.
>     4. Stir gently and place the lid on the press, but don't plunge yet.
>     5. Let the coffee steep for 4 minutes.
>     6. Slowly press the plunger down to separate the grounds from the brewed coffee.
>     7. Pour and enjoy!
> 
> **3. Pour Over:**
> 
> * **What you need:** Pour over cone, filter, medium-fine ground coffee, hot water.
> * **How to do it:**
>     1. Place a filter in the cone and rinse it with hot water to remove any paper taste.
>     2. Add medium-fine ground coffee to the filter.
>     3. Slowly pour hot water over the coffee grounds in a circular motion, making sure all grounds are saturated.
>     4. Let the coffee drip through into your cup.
>     5. Enjoy!
> 
> **4. Cold Brew:**
> 
> * **What you need:** Coarse ground coffee, cold water, container.
> * **How to do it:**
>     1. Combine coarse ground coffee and cold water in a container (ratio of 1:8 coffee to water).
>     2. Stir well and cover the container.
>     3. Let the coffee steep in the refrigerator for 12-24 hours.
>     4. Strain the coffee through a fine-mesh sieve or cheesecloth.
>     5. Serve over ice and enjoy!
> 
> **Tips for Better Coffee:**
> 
> * **Use fresh, high-quality beans:** The better the beans, the better the coffee.
> * **Grind your beans right before brewing:** This preserves the flavor and aroma.
> * **Use filtered water:** Impurities in water can affect the taste of your coffee.
> * **Experiment with different brewing methods:** Find the method that you enjoy the most.
> * **Clean your coffee maker regularly:** This will help to prevent buildup and ensure that your coffee tastes its best.
> 
> 
> 
> Let me know if you have any other questions or want to explore a specific brewing method in more detail!<end_of_turn>
> <end_of_turn>
> 
> <start_of_turn>user
> what is the best coffee?<end_of_turn>
> 
> 
> Ah, the eternal question! There's no single "best" coffee because taste is subjective. What one person finds delicious, another might find bitter or bland. 
> 
> However, I can give you some pointers to help you find YOUR best coffee:
> 
> **Factors that influence coffee taste:**
> 
> * **Bean Origin:** Different regions produce beans with unique flavor profiles. 
>     * **Latin America:** Often known for chocolatey, nutty, and smooth flavors. (Colombia, Brazil, Guatemala)
>     * **Africa:** Can have bright acidity, fruity notes, and floral aromas. (Ethiopia, Kenya, Tanzania)
>     * **Asia/Pacific:** Often earthy, spicy, and full-bodied. (Indonesia, Vietnam)
> * **Roast Level:**
>     * **Light:** More acidic, delicate flavors, higher caffeine.
>     * **Medium:** Balanced acidity and body, versatile.
>     * **Dark:** Bold, smoky, bittersweet flavors, lower caffeine.
> * **Brewing Method:**  Each method extracts flavors differently, impacting the final taste.
> 
> **Finding YOUR Best:**
> 
> 1. **Explore Different Origins:** Try coffees from various regions to discover what flavor profiles you enjoy.
> 2. **Experiment with Roast Levels:**  Sample light, medium, and dark roasts to see your preference.
> 3. **Consider Your Brewing Method:**  Different beans work better with certain brewing methods. Ask your local coffee shop for recommendations.
> 4. **Read Reviews:** Online reviews and coffee blogs can offer insights into specific beans and roasters.
> 5. **Visit a Local Coffee Shop:** Talk to baristas, ask for recommendations, and try different coffees.
> 
> **Remember:** Coffee is a journey of discovery! Enjoy the process of exploring and finding your perfect cup.<end_of_turn>
> <end_of_turn>
> 
> <start_of_turn>user
> what are similar drinks?<end_of_turn>
> 
> 
> Here are some drinks similar to coffee, categorized by what aspect of coffee they resemble:
> 
> **Similar Caffeine Kick:**
> 
> * **Tea:**  Wide variety of flavors and caffeine levels, from delicate green tea to robust black tea.
> * **Matcha:**  Powdered green tea with a unique earthy flavor and higher caffeine content than regular green tea.
> * **Yerba Mate:**  South American herbal tea with a slightly bitter, grassy flavor and moderate caffeine.
> * **Energy Drinks:**  High caffeine content, often with added sugar and other stimulants.
> 
> **Similar Flavor Profiles:**
> 
> * **Chocolate:**  Rich, bittersweet flavors can complement coffee's roasted notes.
> * **Caramel:**  Sweet and buttery, caramel adds a creamy dimension similar to coffee's body.
> * **Vanilla:**  A classic pairing with coffee, vanilla adds a warm, comforting sweetness.
> * **Espresso-Based Drinks:**  Lattes, cappuccinos, macchiatos, and mochas all feature espresso as a base, offering variations in milk and flavorings.
> 
> **Similar Ritual and Experience:**
> 
> * **Hot Chocolate:**  A comforting and warming drink often enjoyed in the morning or evening.
> * **Herbal Teas:**  A relaxing and flavorful alternative to coffee, enjoyed for their health benefits and calming effects.
> * **Golden Milk:**  A turmeric-based drink with

In [11]:
chat.show_history()

<start_of_turn>user
help me understand how to make coffee<end_of_turn>

<start_of_turn>model


<start_of_turn>user
help me understand how to make coffee<end_of_turn>


##  Let's Brew Some Coffee! ☕️

Making coffee is a simple process, but there are many ways to do it. Here's a breakdown of some popular methods:

**1. Drip Coffee Maker:**

* **What you need:** Drip coffee maker, coffee filter, ground coffee, water.
* **How to do it:**
    1. Fill the water reservoir with fresh, cold water.
    2. Place a filter in the filter basket.
    3. Add ground coffee to the filter (follow the coffee maker's instructions for the right amount).
    4. Turn on the coffee maker and let it brew.
    5. Enjoy your freshly brewed coffee!

**2. French Press:**

* **What you need:** French press, coarse ground coffee, hot water.
* **How to do it:**
    1. Heat water to just off the boil (around 200°F).
    2. Add coarse ground coffee to the French press.
    3. Pour hot water over the coffee grounds, maki